In [7]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


# Graph + Warshall

In [67]:
import pandas as pd
import networkx as nx

In [54]:
routes = pd.read_excel('data/Distance.xlsx', sheet_name='Roads', index_col='road_id')

In [59]:
def get_node(route):
    travel_time = ((route.distance / 1000) / 30) * 60
    return tuple([route.source_point_id, route.target_point_id, {'weight': travel_time}])

In [64]:
edges = list(routes.apply(get_node, axis=1).values)

In [65]:
graph = nx.DiGraph(edges)

In [79]:
predecessor, distances = nx.floyd_warshall_predecessor_and_distance(graph)

In [97]:
all_points = list(set(routes.source_point_id) | set(routes.target_point_id))

In [111]:
warshall_map = {}

for source_point in all_points:
    warshall_map[source_point] = {}
    warshall_map[source_point]['distance'] = {}
    warshall_map[source_point]['route'] = {}
    
    for destination_point in all_points:
        warshall_map[source_point]['distance'][destination_point] = distances[source_point][destination_point]
        warshall_map[source_point]['route'][destination_point] = nx.reconstruct_path(source_point, destination_point, predecessor)

# Vodila Generator

In [7]:
default_spawnpoint_id = 4
bus_capacities = [100] * 30 + [50] * 10

In [8]:
drivers = []
for driver_id, bus_capacity in enumerate(bus_capacities):
    drivers.append({
        'id': driver_id + 1,
        'location': default_spawnpoint_id,
        'task_id': None,
        'capacity': bus_capacity,
        'free_time': None
    })